### AB Model from spec

Package installation

In [1]:
# !pip3 install seaborn
# !pip3 install python-statemachine
# !pip3 install mesa
# !pip3 install transitions
# !pip3 install scipy
# !pip3 install cufflinks
# !pip3 install graphviz pygraphviz
# !pip3 install graphviz
# !pip3 install transitions[diagrams]
# !pip3 uninstall mesa

Package importation

In [2]:
# imports
import os
import seaborn as sns
from random import choice
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import mesa
from mesa import Agent, Model
from mesa.time import RandomActivation, RandomActivationByType, SimultaneousActivation
from mesa.datacollection import DataCollector
from matplotlib import pyplot as plt, patches
import scipy.stats as ss
import cufflinks as cf
cf.go_offline()
from plotly.offline import iplot
from transitions import Machine
import random
from transitions.extensions import GraphMachine
import graphviz
import timeit
import timeit

import logging
os.environ["PATH"] += os.pathsep + '/Users/isaacao/homebrew/bin' # for graphviz

Model component importation

In [3]:
from EV.agent import EV, ChargeStation
import EV.model as model
from EV.statemachine import EVSM
from EV.model import get_evs_charge, get_evs_charge_level, get_evs_active, get_evs_charging, get_evs_queue, get_evs_travel, get_evs_not_idle, get_active_chargestations, get_eod_evs_socs, get_evs_destinations, get_ev_distance_covered

State diagram

### Model

Model parameters

In [4]:
ticks =  48 # 3600 ticks = 3600 seconds = 1 hour
no_evs = 3
# no of css must be fixed
no_css = 5

Run Model

In [10]:
model_run = model.EVModel(ticks=ticks, no_evs=no_evs, no_css=no_css)
for i in range(ticks):
    model_run.step()


EV info: ID: 0, destination name: City C, journey type: InterUrban, max_battery: 41, speed: 20, State: Idle.
EV info (Cont'd): Start time: 9, distance goal: 185, energy consumption rate 0.5.

EV info: ID: 1, destination name: City C, journey type: InterUrban, max_battery: 68, speed: 20, State: Idle.
EV info (Cont'd): Start time: 8, distance goal: 185, energy consumption rate 0.5.

EV info: ID: 2, destination name: friend_1, journey type: Urban, max_battery: 51, speed: 10, State: Idle.
EV info (Cont'd): Start time: 13, distance goal: 30, energy consumption rate 0.2.

CP info: ID: 3, initialized. Charge rate: 7.5 kW.

CP info: ID: 4, initialized. Charge rate: 7.5 kW.

CP info: ID: 5, initialized. Charge rate: 7.5 kW.

CP info: ID: 6, initialized. Charge rate: 7.5 kW.

CP info: ID: 7, initialized. Charge rate: 7.5 kW.
Charging Station: 3 is at checkpoint: 40 miles.
Charging Station: 4 is at checkpoint: 80 miles.
Charging Station: 5 is at checkpoint: 120 miles.
Charging Station: 6 is at c

In [14]:
run_stats = model_run.datacollector.get_model_vars_dataframe()
print(run_stats)

    EVs Charging  EVs Activated  EVs Travelling  EVs Queued  \
0              0              3               0           0   
1              0              3               0           0   
2              0              3               0           0   
3              0              3               0           0   
4              0              3               0           0   
5              0              3               0           0   
6              0              3               0           0   
7              0              3               0           0   
8              0              3               1           0   
9              0              3               2           0   
10             0              3               2           0   
11             0              3               2           0   
12             1              3               1           0   
13             2              3               1           0   
14             2              3               1        

In [ ]:
a = 26 % 24
print(a) 

Export results to CSV file

In [ ]:
model_run.datacollector.get_model_vars_dataframe().to_csv('100agent_model_output.csv')

### Batching

In [ ]:
from mesa.batchrunner import BatchRunner

EVcounts = (100,500,1000)
cpcounts = (1,2)
tickcounts = (24,48)
model_reporters={'EVs Charged': get_evs_charged,
                'EVs Activated': get_evs_active,
                'EVs Travelling': get_evs_travel,
                'EVs Charge Level': get_evs_charge_level,
                'EVs Currently charging': get_evs_charging,
                'EVs Not Idle': get_evs_not_idle,
                'EOD Battery SOC': get_eod_evs_socs,
                'EVs Destinations': get_evs_destinations,
                }
# parameters = {"no_evs": range(1000,20000,3000), "no_cps": 1}
parameters = {"no_evs": EVcounts, "no_cps": cpcounts, "ticks": tickcounts}
batch_run = BatchRunner(model.EVModel, parameters, max_steps=24, iterations=1, model_reporters= model_reporters) #iterations=1
batch_run.run_all()

In [ ]:
# batch_df = batch_run.get_model_vars_dataframe()

In [ ]:
# print(batch_df)

### Visualisations

### Scrapbook